In [1]:
from PIL import Image
from feature_extractor import FeatureExtractor
from pathlib import Path
import numpy as np
from elasticsearch import Elasticsearch
from os import listdir
from os.path import isfile, join
import argparse
import json
import pathlib
from elasticsearch.helpers import bulk

# Creating index

In [2]:
es = Elasticsearch([{'host': 'localhost', 'port': 9200, 'scheme': 'http'}])

In [3]:
es.cluster.health(wait_for_status='yellow', request_timeout=1)

C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_8984\1636688961.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.cluster.health(wait_for_status='yellow', request_timeout=1)
C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_8984\1636688961.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.cluster.health(wait_for_status='yellow', request_timeout=1)


ObjectApiResponse({'cluster_name': 'elasticsearch', 'status': 'yellow', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 16, 'active_shards': 16, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 1, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 4, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 3802985, 'active_shards_percent_as_number': 94.11764705882352})

In [4]:
index = 'bdimages'

In [108]:
es.options(ignore_status = [400, 404]).indices.delete(index = index)

C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_6116\4094657150.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.options(ignore_status = [400, 404]).indices.delete(index = index)


ObjectApiResponse({'acknowledged': True})

In [110]:
settings = {
  "settings": {
    "elastiknn": True,
    "number_of_shards": 1,
    "number_of_replicas": 0
  }
}

mapping = {
  "dynamic": False,
  "properties": {
      "feature_vector": {
          "type": "elastiknn_dense_float_vector",
          "elastiknn": {
            "dims": 4096,
            "model": "lsh",
            "similarity": "l2",
            "L": 60,
            "k": 3,
            "w": 2
          }
    },
    "image_path":{"type":"text","index":False}
  }
}

created_index = es.indices.create(
    index = index,
    body = settings,
    ignore = 400
)

C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_6116\3605046962.py:27: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  created_index = es.indices.create(
C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_6116\3605046962.py:27: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  created_index = es.indices.create(
C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_6116\3605046962.py:27: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  created_index = es.indices.create(


In [5]:
fe = FeatureExtractor()

# Indexing Images

In [104]:
my_path = "../bdimage/Objects/"

In [105]:
class_dir = [f for f in listdir(my_path) if join(my_path, f)] #Different classes in the Objects folder

In [111]:
img_nb = 1
for clas in class_dir :
    classpath = my_path + clas + '/'
    img_files = [f for f in listdir(classpath) if isfile(join(classpath, f))]
    for _file in img_files:	
        print(_file) 
        image = Image.open(classpath + _file) ##FIX ME PATH
        feature = fe.extract(image) #Returns Ndarray of size 4096 
        # create a new dict obj for the Elasticsearch doc
        _source = {}
        _source["image_path"] = 'Objects/' + clas + "/" + _file    	
        # put the encoded string into the _source dict
        _source["feature_vector"] = feature

        _id = img_nb
        # call the Elasticsearch client's index() method
        resp = es.index(
            index = index,
            id = _id,
            #doc_type = '_doc',
                body = _source,
                request_timeout=60)

                                        
        print ("\nElasticsearch index() response -->", resp)
        img_nb += 1

image_0001.jpg
1/1 [==============================] - 0s 479ms/step


C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_6116\2365289595.py:17: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  resp = es.index(
C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_6116\2365289595.py:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  resp = es.index(
C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_6116\2365289595.py:17: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  resp = es.index(



Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '1', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 0, '_primary_term': 1}
image_0002.jpg
1/1 [==============================] - 0s 179ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '2', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 1, '_primary_term': 1}
image_0003.jpg
1/1 [==============================] - 0s 167ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '3', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1}
image_0004.jpg
1/1 [==============================] - 0s 163ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '4', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successf

1/1 [==============================] - 0s 160ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '31', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 30, '_primary_term': 1}
image_0032.jpg
1/1 [==============================] - 0s 161ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '32', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 31, '_primary_term': 1}
image_0033.jpg
1/1 [==============================] - 0s 166ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '33', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 32, '_primary_term': 1}
image_0034.jpg
1/1 [==============================] - 0s 166ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '34', '_version

1/1 [==============================] - 0s 173ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '60', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 59, '_primary_term': 1}
image_0019.jpg
1/1 [==============================] - 0s 174ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '61', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 60, '_primary_term': 1}
image_0023.jpg
1/1 [==============================] - 0s 167ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '62', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 61, '_primary_term': 1}
image_0031.jpg
1/1 [==============================] - 0s 194ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '63', '_version

...

1/1 [==============================] - 1s 714ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7368', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7367, '_primary_term': 1}
image_0034.jpg
1/1 [==============================] - 0s 467ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7369', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7368, '_primary_term': 1}
image_0035.jpg
1/1 [==============================] - 1s 705ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7370', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7369, '_primary_term': 1}
image_0036.jpg
1/1 [==============================] - 1s 700ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '73

1/1 [==============================] - 0s 469ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7397', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7396, '_primary_term': 1}
image_0007.jpg
1/1 [==============================] - 1s 704ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7398', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7397, '_primary_term': 1}
image_0008.jpg
1/1 [==============================] - 1s 701ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7399', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7398, '_primary_term': 1}
image_0009.jpg
1/1 [==============================] - 1s 539ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '74

1/1 [==============================] - 1s 581ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7426', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7425, '_primary_term': 1}
image_0036.jpg
1/1 [==============================] - 0s 432ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7427', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7426, '_primary_term': 1}
image_0037.jpg
1/1 [==============================] - 1s 743ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7428', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7427, '_primary_term': 1}
image_0038.jpg
1/1 [==============================] - 1s 741ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '74

1/1 [==============================] - 0s 462ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7455', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7454, '_primary_term': 1}
image_0026.jpg
1/1 [==============================] - 0s 470ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7456', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7455, '_primary_term': 1}
image_0027.jpg
1/1 [==============================] - 0s 455ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7457', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7456, '_primary_term': 1}
image_0028.jpg
1/1 [==============================] - 0s 467ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '74

1/1 [==============================] - 1s 535ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7484', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7483, '_primary_term': 1}
image_0055.jpg
1/1 [==============================] - 1s 717ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7485', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7484, '_primary_term': 1}
image_0056.jpg
1/1 [==============================] - 1s 656ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7486', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7485, '_primary_term': 1}
image_0057.jpg
1/1 [==============================] - 1s 745ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '74

In [6]:
my_path = "../bdimage/Texture/"
class_dir = [f for f in listdir(my_path) if join(my_path, f)] #Different classes in the Texture folder
img_nb = 7491 #Increment the last id
for clas in class_dir :
    classpath = my_path + clas + '/'
    img_files = [f for f in listdir(classpath) if isfile(join(classpath, f))]
    for _file in img_files:	
        print(_file) 
        image = Image.open(classpath + _file) ##FIX ME PATH
        feature = fe.extract(image) #Returns Ndarray of size 4096 
        # create a new dict obj for the Elasticsearch doc
        _source = {}
        _source["image_path"] = 'Texture/' + clas + "/" + _file    	

        # convert the nested Python np.ndarray with the shape=(4096, ) to a str
        # put the encoded string into the _source dict
        _source["feature_vector"] = feature

        _id = img_nb
        # call the Elasticsearch client's index() method
        resp = es.index(
            index = index,
            id = _id,
            #doc_type = '_doc',
                body = _source,
                request_timeout=60)

                                        
        print ("\nElasticsearch index() response -->", resp)
        img_nb += 1

Bark.0000.03.jpg
1/1 [==============================] - 0s 305ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7491', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7490, '_primary_term': 1}
Bark.0002.10.jpg


C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_8984\908683378.py:21: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  resp = es.index(
C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_8984\908683378.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  resp = es.index(
C:\Users\KaryGauss\AppData\Local\Temp\ipykernel_8984\908683378.py:21: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  resp = es.index(


1/1 [==============================] - 0s 163ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7492', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7491, '_primary_term': 1}
Bark.0003.04.jpg
1/1 [==============================] - 0s 167ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7493', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7492, '_primary_term': 1}
Bark.0003.09.jpg
1/1 [==============================] - 0s 160ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7494', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7493, '_primary_term': 1}
Bark.0003.16.jpg
1/1 [==============================] - 0s 158ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id

1/1 [==============================] - 0s 201ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7521', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7520, '_primary_term': 1}
Bush.0012.03.jpg
1/1 [==============================] - 0s 184ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7522', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7521, '_primary_term': 1}
Bush.0014.03.jpg
1/1 [==============================] - 0s 171ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7523', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7522, '_primary_term': 1}
Bush.0014.12.jpg
1/1 [==============================] - 0s 191ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id

1/1 [==============================] - 0s 157ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7550', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7549, '_primary_term': 1}
Fabric.0030.13.jpg
1/1 [==============================] - 0s 183ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7551', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7550, '_primary_term': 1}
Fabric.0031.10.jpg
1/1 [==============================] - 0s 164ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '7552', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 7551, '_primary_term': 1}
Fabric.0033.08.jpg
1/1 [==============================] - 0s 168ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc'

...

1/1 [==============================] - 0s 157ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9612', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9611, '_primary_term': 1}
Technic.0009.13.jpg
1/1 [==============================] - 0s 156ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9613', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9612, '_primary_term': 1}
Technic.0009.14.jpg
1/1 [==============================] - 0s 156ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9614', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9613, '_primary_term': 1}
Technic.0009.16.jpg
1/1 [==============================] - 0s 157ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_d

1/1 [==============================] - 0s 154ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9640', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9639, '_primary_term': 1}
Technic.0013.07.jpg
1/1 [==============================] - 0s 163ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9641', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9640, '_primary_term': 1}
Technic.0013.08.jpg
1/1 [==============================] - 0s 159ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9642', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9641, '_primary_term': 1}
Technic.0013.11.jpg
1/1 [==============================] - 0s 158ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_d

1/1 [==============================] - 0s 173ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9669', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9668, '_primary_term': 1}
Tiles.0002.12.jpg
1/1 [==============================] - 0s 166ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9670', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9669, '_primary_term': 1}
Tiles.0002.14.jpg
1/1 [==============================] - 0s 167ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9671', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9670, '_primary_term': 1}
Tiles.0003.03.jpg
1/1 [==============================] - 0s 171ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '

1/1 [==============================] - 0s 158ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9698', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9697, '_primary_term': 1}
Tire.0002.13.jpg
1/1 [==============================] - 0s 162ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9699', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9698, '_primary_term': 1}
Tire.0002.15.jpg
1/1 [==============================] - 0s 156ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9700', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9699, '_primary_term': 1}
Tire.0002.16.jpg
1/1 [==============================] - 0s 158ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id

1/1 [==============================] - 0s 163ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9727', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9726, '_primary_term': 1}
Tree.0001.11.jpg
1/1 [==============================] - 0s 156ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9728', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9727, '_primary_term': 1}
Tree.0001.12.jpg
1/1 [==============================] - 0s 169ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id': '9729', '_version': 1, 'result': 'created', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 9728, '_primary_term': 1}
Tree.0002.02.jpg
1/1 [==============================] - 0s 169ms/step

Elasticsearch index() response --> {'_index': 'bdimages', '_type': '_doc', '_id